In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
import joblib

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



# Features:

## Volume and Duration
1. Breadth (may not be relevant) - number of transactions in an interval
2. VolumeAll: the total number of shares transacted in the interval
3. VolumeAvg: the avg number of shares transacted in the interval
4. VolumeMax: maximum num of shares transacted in one transaction in the interval.

## Return and Imbalance:
1. Lambda: 

## Exploratory Data Analysis
The training data contain 481 days, covering the period of 3:50pm to 3:59pm at the Nasdaq

In [21]:
# train.date_id.unique()

In [22]:
train[["matched_size", "imbalance_size", "imbalance_buy_sell_flag"]]

train[train.matched_size == 0]

train["matched_ratio"] = train.imbalance_buy_sell_flag*train.imbalance_size/train.matched_size
train["test_ratio"] = train.imbalance_buy_sell_flag*train.matched_size/train.imbalance_size

In [23]:
# train.loc[(train.stock_id == 0) & (train.date_id == 0) & (train.seconds_in_bucket == 0)]
# train[["seconds_in_bucket", "wap", "stock_id", "date_id", "seconds_in_bucket", "matched_size", "imbalance_size", "imbalance_buy_sell_flag", "matched_ratio"]]

train[["seconds_in_bucket", "far_price", "stock_id", "date_id"]].dropna().seconds_in_bucket.unique()

array([300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420,
       430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540])

Note Reference price is for all seconds but far price is only for seconds_in_bucket 300-540.
In other words for 4:55-4:59

In [24]:
# stock_id = train[train.matched_ratio.isna()].stock_id.unique()
# train[train.stock_id.isin(stock_id)].seconds_in_bucket.unique()

In [25]:
# Exploring features

# train

here we can see that some stocks don't have an auction

In [26]:

def make_data(df, training=False, full_data=False):
	data = df.copy()
	# data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap_logreturns", "matched_ratio"]

	# data["wap_logreturns"] = np.log(data.wap/data.wap.shift(1))

	data["prev_wap"] = data.groupby(["date_id", "stock_id"]).wap.shift(1)

	# data["prev_wap"] = data["prev_wap"].fillna(0)

	# data["wap_logreturns"] = np.log(data.wap / data.prev_wap)

	data["PastReturn"] = 1 - data.eval('wap /prev_wap')

	data["matched_ratio"] = data.imbalance_buy_sell_flag*data.imbalance_size/data.matched_size

	# data["wap_man"] = data.eval("(bid_price*ask_size + ask_price*bid_size)/(bid_size + ask_size)")

	data["imb_s1"] = data.eval('(bid_size - ask_size)/(bid_size + ask_size)')
	data["imb_s2"] = data.eval('(imbalance_size - matched_size)/(matched_size + imbalance_size)')

	data["VolumeAll"] = data.eval('bid_size + ask_size') # explore

	data["lambda"] = data.eval('(bid_price - ask_price)/VolumeAll')

	data["LobImbalance"] = data.eval('(ask_size - bid_size)/VolumeAll')
	data["TxnImbalance"] = data.eval('(imbalance_size - matched_size)/VolumeAll')



	# data["matched_ratio"] = data["matched_ratio"].fillna(0)

	# data["wap_logreturns"] = data["wap_logreturns"].fillna(0)

	# data.replace([np.inf, -np.inf], np.nan, inplace=True)
	# data['ewa6'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=6, adjust=False).mean()).values
	# data['ewa12'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=12, adjust=False).mean()).values

	if full_data == True:
		return data

	data = data.fillna(0)
	features = ["PastReturn", "lambda", "LobImbalance", "matched_ratio", "imb_s1", "imb_s2", "imbalance_buy_sell_flag"]
	if training:
		X = data[features].to_numpy()
		y = data.target.to_numpy()
		# y = np.array(data.target).reshape(-1,1)
		return X[np.isfinite(X).all(1)], y[np.isfinite(X).all(1)]
		# return X, y

	X = data[features].copy()

	# X.wap_logreturns = X.wap_logreturns.fillna(0)
	# X.matched_ratio = X.matched_ratio.fillna(0)
	# X.fillna(0,inplace=True)



	return X.to_numpy()


In [27]:
df = make_data(train, full_data=True)

df[(df.matched_size - df.imbalance_size < 0)]

df.eval("(matched_size - imbalance_size)").abs()

0          10199673.95
1           1475610.34
2           1516488.16
3           6472063.35
4          17413064.99
              ...     
5237975    25839638.85
5237976     8838188.64
5237977    12725436.10
5237978    93772372.21
5237979    22189391.61
Length: 5237980, dtype: float64

# Hyperparameter Tuning
We separate these 480 days into 5 pieces perform a 5-fold cross-validation on them.

In [28]:
import getpass
X, y = make_data(train, training=True)

from tune import TuningSession

if getpass.getuser() == "vinicius":
    hyperparam_dists_path = '/Users/vinicius/Projects/kaggle/kaggle-optiver/kaggle_optiver/hyperparameters.yaml'
elif getpass.getuser == "ephraimsutherland":
    hyperparam_dists_path = '/Users/ephraimsutherland/Documents/kaggle-optiver/kaggle_optiver/hyperparameters.yaml'

ts = TuningSession(hyperparam_dists_path=hyperparam_dists_path)

# X_train, X_test, y_train, y_test = train_test_split(X,y ,
#                                    random_state=104,
#                                    test_size=0.25,
#                                    shuffle=True)

In [29]:
ts.run(data=X, labels=y)

[17:59:11] Preparing and Transforming data
[17:59:11] Starting LGBMRegressor's study


[I 2023-12-19 17:59:11,794] A new study created in memory with name: no-name-82dd456a-26c3-40c9-b92f-f4cea5b824a4


[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.039076
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.046583
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.054309
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.049900
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.047935


[I 2023-12-19 17:59:17,025] Trial 0 finished with value: 6.333961677505403 and parameters: {'n_estimators': 8, 'max_depth': 10, 'num_leaves': 128, 'learning_rate': 0.6000000000000001, 'reg_lambda': 0.15000000000000002, 'boosting_type': 'gbdt', 'subsample': 0.9000000000000001, 'colsample_bytree': 0.8, 'reg_alpha': 0.7000000000000001, 'force_row_wise': True, 'n_jobs': -1, 'random_state': 42}. Best is trial 0 with value: 6.333961677505403.


[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.039076
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.046583
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.054309
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.049900
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.047935


...

[I 2023-12-19 17:59:54,079] Trial 7 finished with value: 6.333121467555759 and parameters: {'n_estimators': 16, 'max_depth': 5, 'num_leaves': 16, 'learning_rate': 0.8500000000000001, 'reg_lambda': 0.7000000000000001, 'boosting_type': 'dart', 'subsample': 0.3, 'colsample_bytree': 0.6000000000000001, 'reg_alpha': 0.1, 'force_row_wise': True, 'n_jobs': -1, 'random_state': 42}. Best is trial 2 with value: 6.330827071300677.


[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.039076
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.046583
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.054309
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.049900
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.047935


[I 2023-12-19 18:00:10,064] Trial 8 finished with value: 6.334074996528038 and parameters: {'n_estimators': 20, 'max_depth': 8, 'num_leaves': 64, 'learning_rate': 0.1, 'reg_lambda': 0.30000000000000004, 'boosting_type': 'dart', 'subsample': 0.8, 'colsample_bytree': 1.0, 'reg_alpha': 0.45, 'force_row_wise': True, 'n_jobs': -1, 'random_state': 42}. Best is trial 2 with value: 6.330827071300677.


[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.039076
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.046583
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.054309
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.049900
[LightGBM] [Info] Total Bins 1533
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 7
[LightGBM] [Info] Start training from score -0.047935


[I 2023-12-19 18:00:14,076] Trial 9 finished with value: 6.346415558779484 and parameters: {'n_estimators': 4, 'max_depth': 8, 'num_leaves': 128, 'learning_rate': 0.6000000000000001, 'reg_lambda': 0.8, 'boosting_type': 'dart', 'subsample': 0.6000000000000001, 'colsample_bytree': 0.4, 'reg_alpha': 0.1, 'force_row_wise': True, 'n_jobs': -1, 'random_state': 42}. Best is trial 2 with value: 6.330827071300677.
[18:00:14] Finished LGBMRegressor's study
[18:00:14] Finished


In [35]:
trials_LGBM = ts.trials_dict['LGBMRegressor']
trials_LGBM.sort_values('value')[:len(trials_LGBM)//10]

,number,value,datetime_start,datetime_complete,duration,params_boosting_type,params_colsample_bytree,params_force_row_wise,params_learning_rate,params_max_depth,params_n_estimators,params_n_jobs,params_num_leaves,params_random_state,params_reg_alpha,params_reg_lambda,params_subsample,state
2,2,6.330827,2023-12-19 17:59:21.289331,2023-12-19 17:59:26.157406,0 days 00:00:04.868075,gbdt,0.8,True,0.4,4,16,-1,48,42,0.0,0.45,0.7,COMPLETE


In [10]:
def models(model, i, training=False):
	if training==True:
		model.fit(X,y)
		joblib.dump(model, './models/model_{i}.model')



In [11]:
model1 = LinearRegression()
model2 = lgb.LGBMRegressor(objective='regression_l1', n_estimators=500)

# models(model1)

for i,model in enumerate([model1]):
	# print(i)
	models(model, i, training=True)


In [12]:
# train[train.seconds_in_bucket > 10].head()
# train.loc[train.stock_id==1,['target', 'time_id', 'stock_id', 'seconds_in_bucket']]
train[train.stock_id==1]['seconds_in_bucket'].unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

In [13]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


In [14]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    X = make_data(test)
    # sample_prediction['target'] = model.predict(X)
    sample_prediction['target'] = model1.predict(X)
    # sample_prediction['target'] = 0.5
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [15]:
!tail submission.csv
# sub = pd.read_csv("./example_test_files/sample_submission.csv")
sub = pd.read_csv("submission.csv")

test = pd.read_csv("./example_test_files/test.csv")

revealed = pd.read_csv("./example_test_files/revealed_targets.csv")


# test.merge

tail: submission.csv: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'

In [14]:
sub = sub[sub.target != "target"]
sub.target = sub.target.astype(float)

NameError: name 'sub' is not defined

In [13]:
ans = train[["row_id", "target"]].merge(sub, on="row_id", how="right")

NameError: name 'sub' is not defined

In [18]:
mae = np.mean(np.abs(ans.target_x - ans.target_y))
mae

5.2774822936432155

Original Naive Estimate: 5.27941

With bid-ask size imbalance: 5.279170945560908

As above but adding imbalance-matched difference ratio: 5.278630645573058


Same as above but using lgb: 5.278595988137708